In [10]:
from PIL import Image
import os
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Activation, BatchNormalization, concatenate, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.models import load_model
from sklearn.metrics import classification_report

In [2]:
#이미지 dir 경로 풀러오기 (코드와 이미지 폴더가 같은 폴더 안에 있어야 됨)
image_dir_path = os.path.dirname(os.path.abspath("__file__")) + '/image'
categories = os.listdir('./image')
select_cat = []
nb_classes = 0
mnist = tf.keras.datasets.mnist
image_w = 224
image_h = 224

x = []
y = []
#이미지 몇개 이상 분포만 불러오는 부분
count = 0;
for idx, c in enumerate(categories):
    dir = image_dir_path + '/' + c
    f = glob(dir+'/*.jpg')
    
    if len(f) >= 200:
        nb_classes += 1
        select_cat.append(dir)
print(select_cat)

['C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Bu_anil', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Chwi_gajil', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Eeo_eojosa', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Ga_olheul', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Gyeong_byeoseul', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Ha_alae', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Ha_eojji', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Il_han', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/I_sseo', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Ji_al', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Ji_gal', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Mu_eobseul', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Wi_hal', 'C:\\Users\\HYEONGCHEOL\\Desktop\\TestFile/image/Yu_isseul']


In [3]:
#이미지 데이터 불러오고 라벨링
for index, cat in enumerate(select_cat):
    files = glob(select_cat[index]+'/*.jpg')
    for i, f in enumerate(files):
            img = Image.open(f)
            img = img.convert("RGB")
            img = img.resize((image_w, image_h))
            data = np.asarray(img)
            
            x.append(data)
            y.append(index)

In [4]:
#이미지 행렬화 및 확인
x = np.array(x)
y = np.array(y)
print(x.shape)
print(y.shape)

(3546, 224, 224, 3)
(3546,)


In [5]:
#트레인 테스트 데이터 스플릿(8:2)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)
xy = (x_train, x_test, y_train, y_test)
x_train = x_train.astype(float) / 255
x_test = x_test.astype(float) / 255

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(2836, 224, 224, 3) (2836,)
(710, 224, 224, 3) (710,)


In [6]:
growth_rate = 16
theta = 0.5

def DenseBlock(x, num):
    layer_list = [x]
    for i in range(num):
        x1 = BatchNormalization()(x)
        x1 = Activation('relu')(x1)
        x1 = Conv2D(growth_rate*4, kernel_size=1, padding='same', use_bias=False)(x1)
        x1 = BatchNormalization()(x1)
        x1 = Activation('relu')(x1)
        x1 = Conv2D(growth_rate, kernel_size=3, padding='same', use_bias=False)(x1)
        layer_list.append(x1)
        x = concatenate(layer_list, axis=-1) 
    return x


def Transition_layer(x, name):
    num_ch = int(x.get_shape().as_list()[-1] * theta) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_ch, kernel_size=3, padding='same', use_bias=False)(x)
    x = AveragePooling2D(pool_size=2, strides=2, padding='same', name=name)(x)
    
    return x


inputs = Input(shape=(224, 224, 3))
x = Conv2D(64, kernel_size=7, strides=2, padding='same', use_bias=False)(inputs) # (None,112, 112, 64)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x) # (None,56, 56, 64)

x = DenseBlock(x, 3) # (None,56, 56, 256)
x = Transition_layer(x, 'transition_layer_1') # (None,28, 28, 128)
x = DenseBlock(x, 6) # (None,28, 28, 512)
x = Transition_layer(x, 'transition_layer_2') # (None,14, 14, 256)
x = DenseBlock(x, 12) # (None,14, 14, 1024)
x = Transition_layer(x, 'transition_layer_3') # (None,7, 7, 512)
x = DenseBlock(x, 8) # (None,7, 7, 1024)
x = GlobalAveragePooling2D()(x) # (None,1024)
outputs = Dense(14, activation='softmax')(x) # (None, 14)

In [7]:
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [9]:
num_classes = 14
y_train_one_hot = to_categorical(y_train, num_classes)
y_test_one_hot = to_categorical(y_test, num_classes)

In [ ]:
#모델 학습(Adam)
model.compile('Adam', loss='categorical_crossentropy', metrics=['acc'])
history_Adam = model.fit(x_train, y_train, batch_size=32, epochs=20, validation_split=0.2)
#모델 학습2(SGD)
model.compile('SGD', loss='categorical_crossentropy', metrics=['acc'])
history_SGD = model.fit(x_train, y_train, batch_size=32, epochs=20, validation_split=0.2)

In [ ]:
#Test 확인
test_loss, test_acc = model.evaluate(x_train,  y_train1, verbose=2)
print('\n테스트 loss:', test_loss)
print('\n테스트 acc', test_acc)

In [ ]:
#모델 저장
model.save('densenet_Comparison_of_cursive_script.h5')

In [ ]:
#Adam 학습 중 loss, acc 그래프
plt.plot(history_Adam.history['loss'],label='train loss')
plt.plot(history_Adam.history['val_loss'], label='val loss')
plt.legend()
plt.show()

plt.plot(history_Adam.history['acc'], label='train acc')
plt.plot(history_Adam.history['val_acc'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
#SGD 학습 중 loss, acc 그래프
plt.plot(history_SGD.history['loss'],label='train loss')
plt.plot(history_SGD.history['val_loss'], label='val loss')
plt.legend()
plt.show()

plt.plot(history_SGD.history['acc'], label='train acc')
plt.plot(history_SGD.history['val_acc'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
#분류 결과 저장 및 one_hot_decoding
y_pred_one_hot = model.predict(x_train)
y_pred = np.argmax(y_pred, axis=1).reshape(-1)

In [ ]:
#모델의  precision    recall  f1-score   값 확인(클래스별 및 평균)
target_names = ['Bu_anil', 'Chwi_gajil', 'Eeo_eojosa', 'Ga_olheul', 'Gyeong_byeoseul', 'Ha_alae', 'Ha_eojji', 'Il_han', 'I_sseo', 'Ji_al', 'Ji_gal', 'Mu_eobseul', 'Wi_hal', 'Yu_isseul']
print(classification_report(y_train, y_pred1, target_names=target_names))